# MOwNiT
## Laboratorium 1 - Arytmetyka komputerowa

Kalkulatory IEEE-754:
* http://babbage.cs.qc.cuny.edu/IEEE-754/
* https://www.h-schmidt.net/FloatConverter/IEEE754.html

In [11]:
# znak, cecha, mantysa
# uwaga - w Julii tablice indeksujemy od 1
# uwaga: ukryta jedynka nie jest pamiętana
# uwaga: cecha jest pamiętana w kodzie z nadmiarem (tutaj BIAS=127)
x = 0.000000000001
decode(x::Float32) = (b=bitstring(x); (b[1], b[2:9], b[10:32]))
b=bitstring(x)
print((b[1], b[2:9], b[10:32]))

('0', "01111010", "11100011001011110011001"

)

## Trochę teorii o IEEE-754

Liczby zmienno przecinkowe są zapisywane w standardzie jako:

$L = (-1)^{znak} * mantysa * 2^{cecha}$ 

* znak - 0 oznacza liczbę dodatnią, 1 ujemną
* mantysa - wpływa na precyzje
* cecha - wpływa na zakres, składająca się z samych 0 albo 1 ma specjalne znaczenie

| Nazwa     | Liczba bitów | Bity znaku | Bity cechy | Bity mantysy | Bias  | Dokładność (cyfry znaczące) | Uwagi                                 |
|-----------|--------------|------------|------------|--------------|-------|-----------------------------|---------------------------------------|
| half      | 16           | 1          | 5          | 10           | 15    | 4                           |                                       |
| single    | 32           | 1          | 8          | 23           | 127   | 7                           |                                       |
| double    | 64           | 1          | 11         | 52           | 1023  | 16                          |                                       |
| quadruple | 128          | 1          | 15         | 112          | 16383 | 34                          | Wprowadzone w 2008, niezbyt popularne |

### Liczby znormalizowane

Mantysa ma wartość z przedziału [1,2) - pierwszy bit ma zawsze wartość 1, więc nie trzeba go zapisywać (jest on tam "domyślnie")
$L = (-1)^{znak} * 1.mantysa * 2^{cecha-bias}$

In [12]:
decode(Float32(2.5))

('0', "10000000", "01000000000000000000000")

Policzenie wartości na podstawie reprezentacji bitowej.

In [13]:
cecha = 2^7-127
mantysa = 2^-2 + 1
cecha, mantysa

(1, 1.25)

In [14]:
L = 1 * mantysa * 2^cecha

2.5

### Liczba 0

Wartości 0 nie da się zapisać w postaci znormalizowanej. Jeżeli zarówno mantysa jak i cecha zawierają same 0, to reprezentowana jest wartość 0.

Ze względu na bit znaku, istnieją dwie możliwości 0: +0 oraz -0.

In [15]:
decode(Float32(-0.0))

('1', "00000000", "00000000000000000000000")

In [16]:
decode(Float32(+0.0))

('0', "00000000", "00000000000000000000000")

In [17]:
Float32(-0.0) == Float32(+0.0) # Oba zera są równe

true

### Liczby zdenormalizowane

Jeżeli cecha składa się z samych 0 to liczba jest *zdenormalizowana* i mantysa nie posiada "domyślnego" bitu. Pozwala to na reprezentacje liczb bliskich 0, które bez denormalizacji byłyby 0.

$L = (-1)^{znak} * 0.mantysa * 2^{-bias+1}$

Uwaga: Liczba 0 nie jest określana jako zdenormalizowana, pomimo, że cecha ma same 0!

In [39]:
L = Float32(1e-42)
decode(Float32(L)), issubnormal(L)

(('0', "00000000", "00000000000001011001010"), true)

### Wartości specjalne

####  Nieskończoność

Reprezentowana jest przez cechę z samych 1 oraz mantysę z samych 0. W zależności od znaku można wyróżnić $-\infty$ oraz $+\infty$.

In [18]:
decode(Inf32)

('0', "11111111", "00000000000000000000000")

In [41]:
decode(-Inf32)

('1', "11111111", "00000000000000000000000")

#### NaN

Reprezentowana jest przez cechę z samych 1 oraz mantysę różną od 0.

`NaN`ami są np. 0/0, $\infty - \infty$

In [19]:
Float32(0)/Float32(0)

NaN32

In [43]:
Inf32 - Inf32

NaN32

In [44]:
decode(NaN32)

('0', "11111111", "10000000000000000000000")

### Praktyczne wskazówki

Kolejność działań ma znaczenie:

In [20]:
e = eps(Float32)/2

5.9604645f-8

In [21]:
(Float32(1.0) + e) + e

1.0f0

In [22]:
Float32(1.0) + (e + e)

1.0000001f0

Wniosek: Sumujemy od najmniejszych

Nie powinno się porównywać floatów bezpośrednio. Większość języków posiada funkcje do porówywania liczb zmiennoprzecinkowych z tolerancją, np. `isapprox` w Julii, `numpy.isclose` w Pythonie etc.

In [48]:
0.1 + 0.2 == 0.3

false

In [49]:
isapprox(0.1 + 0.2, 0.3)

true

## Przykłady

In [50]:
bitstring(Float32(.1))

"00111101110011001100110011001101"

In [51]:
bitstring(Float32(1.0))

"00111111100000000000000000000000"

In [52]:
bitstring(Float64(1.0))

"0011111111110000000000000000000000000000000000000000000000000000"

In [53]:
# domyślnie Float64
bitstring(1.0)

"0011111111110000000000000000000000000000000000000000000000000000"

In [54]:
# Float16
bitstring(Float16(1.0))

"0011110000000000"

In [55]:
println(significand(Float32(1.2)), "*2^", exponent(Float32(1.20)))

1.2*2^0

In [56]:
# co ile liczby sa reprezentowane 
c=Float32[]
a=Float32(1.0)
r=1:10
for i=r
    push!(c, a)
    a=nextfloat(a)
end
map(x-> bitstring(x),c)

10-element Vector{String}:
 "00111111100000000000000000000000"
 "00111111100000000000000000000001"
 "00111111100000000000000000000010"
 "00111111100000000000000000000011"
 "00111111100000000000000000000100"
 "00111111100000000000000000000101"
 "00111111100000000000000000000110"
 "00111111100000000000000000000111"
 "00111111100000000000000000001000"
 "00111111100000000000000000001001"

In [25]:
# Maszynowe epsilon mówi, jaki jest odstęp między kolejnymi liczbami zmiennoprzecinkowymi reprezentowanymi w komputerze
eps(Float32(1))

1.1920929f-7

In [58]:
eps(Float32(2))

2.3841858f-7

In [59]:
# Badanie postaci zdenormalizowanej. Co się dzieje, jak liczby są coraz mniejsze?
a = Float32(1e-33)
while a > 0
    a = a/Float32(2.0)
    println(a, decode(a), "  ", issubnormal(a))
end

5.0e-34('0', "00010000", "01001100010011101001100")  false
2.5e-34('0', "00001111", "01001100010011101001100")  false
1.25e-34('0', "00001110", "01001100010011101001100")  false
6.25e-35('0', "00001101", "01001100010011101001100")  false
3.125e-35('0', "00001100", "01001100010011101001100")  false
1.5625e-35('0', "00001011", "01001100010011101001100")  false
7.8125e-36('0', "00001010", "01001100010011101001100")  false
3.90625e-36('0', "00001001", "01001100010011101001100")  false
1.953125e-36('0', "00001000", "01001100010011101001100")  false
9.765625e-37('0', "00000111", "01001100010011101001100")  false
4.8828126e-37('0', "00000110", "01001100010011101001100")  false
2.4414063e-37('0', "00000101", "01001100010011101001100")  false
1.2207032e-37('0', "00000100", "01001100010011101001100")  false
6.103516e-38('0', "00000011", "01001100010011101001100")  false
3.051758e-38('0', "00000010", "01001100010011101001100")  false
1.525879e-38('0', "00000001", "01001100010011101001100")  false

In [60]:
# Rysowanie wykresów
#using Pkg
#Pkg.add("Plots")
# http://docs.juliaplots.org/latest/tutorial/
using Plots

ArgumentError: ArgumentError: Package Plots not found in current path:
- Run `import Pkg; Pkg.add("Plots")` to install the Plots package.


In [61]:
scatter(1:1:5, [2,2,3,3,5])

UndefVarError: UndefVarError: scatter not defined

## Bonus

Ciekawe zastosowanie znajomości IEEE-754 do obliczania odwrotności pierwiastka, wykorzystane w grze Quake 3: [filmik](https://youtu.be/p8u_k2LIZyo)

# Zadania

### Zadanie 1

Porównać w języku Julia reprezentację bitową liczby 1/3 dla `Float16`, `Float32`, `Float64` oraz liczby, 
która jest inicjalizowana jako `Float16`, a potem rzutowana na `Float64`.

### Zadanie 2

Zbadać, jak zmienia się odległość między kolejnymi liczbami zminnoprzecinkowymi 
reprezentowanymi w komputerze za pomocą języka Julia. 
Narysować wykres używając Plots zależności odległości od wartości liczby dla zakresu  od 1.0 do 1000000.0.

### Zadanie 3

Jedną z bibliotek numerycznych, jaką będziemy używać na zajęciach jest <a href="http://home.agh.edu.pl/~kzajac/dydakt/mownit/lab1/gsl.html">GSL</a> (język C). Korzystając ze <a href="https://www.gnu.org/software/gsl/doc/html/ieee754.html"> wsparcia dla wyświetlania reprezentacji liczb zmiennoprzecinkowych</a> zobaczyć jak zmienia się cecha i mantysa dla coraz mniejszych liczb. Zaobserwować, kiedy matysa przestaje być znormalizowana i dlaczego?

Kod załączyć jako komórka Markdown sformatowana jako C ([link](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#code)). Wynik także jako Markdown (kod albo fragment zrzutu ekranu).

### Zadanie 4

Na przykładzie wybranego algorytmu niestabilnego numerycznie: 
1. Pokazać, że działa źle.
2. Pokazać które konkretnie działania powodują zwiększenie błędu (np. dzielenie przez małą liczbę, cancellation).
3. Zademonstować wersję stabilną.

Wszystkie punkty przedstawić w postaci notatnika Julii.